In [1]:
from pf_simulation import PowerFactorySim as pfsim

FOLDER_NAME = ''
PROJECT_NAME = '39 Bus New England System'
STUDY_CASE_NAME = '1. Power Flow'
MONITORED_VARIABLES = {
    '*.ElmTerm': ['m:u1', 'm:phiu'],
    '*.ElmSym': ['c:firel', 's:outofstep']
}
VAR_FOLDER = {
    '*.ElmTerm': ['magnitude','angle'],
    '*.ElmSym': ['rotor_angle', 'out_of_step']
}

#sim = PowerFactorySim(folder_name=FOLDER_NAME, project_name=PROJECT_NAME, study_case_name=STUDY_CASE_NAME)
sim = pfsim(folder_name=FOLDER_NAME,
            project_name=PROJECT_NAME,
            study_case_name=STUDY_CASE_NAME)
sim.app.Show()

In [2]:
import os
import pandas as pd

# Membuat skenario operasi
f_line = sim.app.GetCalcRelevantObjects('*.ElmLne')
f_location = [0, 20, 50, 80]
f_duration = [0.1, 0.15, 0.18, 0.2, 0.25]
load_level = [i/100 for i in range(50,115,5)]

# Memuat nilai default load dan gen
file_dir = "output/default_data"
load_file = f"{file_dir}/load_data.parquet"
gen_file = f"{file_dir}/generator_data.parquet"
if all(map(os.path.exists, [load_file, gen_file])):
    load_default, gen_default = (pd.read_parquet(load_file), pd.read_parquet(gen_file))
else:
    load_default, gen_default = sim.loadGenData()
    load_default.to_parquet(load_file, index=False)
    gen_default.to_parquet(gen_file, index=False)

# Mengatur parameter untuk simulasi optimal power flow
sim.opfSetup()

# Mengatur parameter beban ke nilai default
sim.loadSetup(1, load_default)
sim.opfAnalysis('los')
sim.ldfAnalysis()

Testing OPF

In [5]:
file_dir = "output/generator_dispatch"
for load_lvl_iter in load_level:
    sim.loadSetup(load_lvl_iter, load_default)
    try:
        sim.opfAnalysis('los')
    except AttributeError:
        print(f'{load_lvl_iter:.2f} load, OPF min los failed, trying min dev')
        sim.opfAnalysis('dev')
    
    # Lakukan analisis daya dan ambil dispatch
    gen_dispatch = sim.ldfAnalysis().gen

    # Simpan hasil ke HDF5
    gen_dispatch.to_parquet(f'{file_dir}/scenario_{int(load_lvl_iter*100)}.parquet', index=False)
    print(f'{load_lvl_iter:.2f} load, OPF and LF running successfully')

0.50 load, OPF and LF running successfully
0.55 load, OPF and LF running successfully
0.60 load, OPF and LF running successfully
0.65 load, OPF and LF running successfully
0.70 load, OPF and LF running successfully
0.75 load, OPF min los failed, trying min dev


KeyboardInterrupt: 

In [5]:
f_loc_iter = f_location[0]
f_line_iter = f_line[0]
f_dur_iter = f_duration[0]
load_lvl_iter = 1

sim.scSetup(f_loc_iter, f_line_iter, f_dur_iter)
sim.rmsSimulation(MONITORED_VARIABLES, t_start=-100, t_step=10, t_stop=30)

In [ ]:
# sim.app.ResLoadData(res)

# # read the results and time steps and store them as lists
# time = []
# var_values = []
# # load results from file
# sim.app.ResLoadData(res)
# # get number of rows (time steps) in the results file
# n_rows = sim.app.ResGetValueCount(res, 0)

# for i in range(n_rows):
#     time.append(sim.app.ResGetData(res, i, -1)[1])

In [28]:
res = sim.app.GetFromStudyCase('All calculations.ElmRes')
sim.app.ResLoadData(res)
res.GetVariable(-1)

'b:tnow'

In [ ]:
res = sim.app.GetFromStudyCase('All calculations.ElmRes')

base_dir = 'D:\Fathin\github\powerfactory_rms\output\dataset'
filename = f'{f_line_iter.loc_name} {f_loc_iter:.0f} percent {f_dur_iter:.2f} s {load_lvl_iter:.2f} load'

elements = list(MONITORED_VARIABLES.keys())

all_var = ['b:tnow']
all_elm = [res]
for element in elements:
    elmObj = sim.app.GetCalcRelevantObjects(element)
    variables = MONITORED_VARIABLES[element]
    i = 0
    for variable in variables:
        output_dir = VAR_FOLDER[element][i]
        full_dir = f"{base_dir}\{filename}"
        all_elm.extend(elmObj)
        all_var.extend([variable]*len(elmObj))
        i += 1


# Export data
comRes = sim.app.GetFromStudyCase("ComRes")
comRes.pResult = res
comRes.iopt_exp = 6 # to export as csv
comRes.f_name = f"{full_dir}.csv"
comRes.iopt_sep = 0 # to use not the system seperator
comRes.col_Sep = r','
comRes.dec_Sep = r'.'
comRes.iopt_honly = 0 # to export data and not only the header
comRes.iopt_csel = 1 # export only selected variables
comRes.iopt_locn = 1
comRes.ciopt_head = 1
comRes.element = all_elm
comRes.iopt_tsel = 1
comRes.to = 30
comRes.cfrom = 0
comRes.variable = all_var
comRes.Execute()

data = pd.read_csv(f"{full_dir}.csv", header=[0, 1])
data.columns = [f"{col[1].split()[0]}_{col[0]}" for col in data.columns]
data

,b:tnow in s_All calculations,m:u1 in p.u._Bus 01,m:u1 in p.u._Bus 02,m:u1 in p.u._Bus 03,m:u1 in p.u._Bus 04,m:u1 in p.u._Bus 05,m:u1 in p.u._Bus 06,m:u1 in p.u._Bus 07,m:u1 in p.u._Bus 08,m:u1 in p.u._Bus 09,...,s:outofstep_G 01,s:outofstep_G 02,s:outofstep_G 03,s:outofstep_G 04,s:outofstep_G 05,s:outofstep_G 06,s:outofstep_G 07,s:outofstep_G 08,s:outofstep_G 09,s:outofstep_G 10
0,0.01,0.709933,0.283960,0.175813,0.419452,0.527684,0.544709,0.545397,0.552322,0.797740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.02,0.704788,0.279243,0.175475,0.419975,0.526552,0.542883,0.545235,0.552532,0.795778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.03,0.700907,0.275990,0.176355,0.423262,0.529152,0.545085,0.548583,0.556044,0.795577,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.04,0.697918,0.273671,0.177817,0.427795,0.533503,0.549205,0.553528,0.561042,0.796326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.05,0.695579,0.271979,0.179541,0.432823,0.538625,0.554197,0.559129,0.566633,0.797587,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,29.96,1.047607,1.054227,1.003824,0.997253,1.005551,1.009557,0.997773,0.996026,1.023082,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2996,29.97,1.047606,1.054225,1.003824,0.997253,1.005551,1.009557,0.997773,0.996026,1.023083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,29.98,1.047605,1.054224,1.003824,0.997253,1.005551,1.009557,0.997773,0.996026,1.023083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,29.99,1.047604,1.054222,1.003824,0.997253,1.005551,1.009557,0.997773,0.996027,1.023084,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def getResult(self, f_loc, f_line, f_dur, load_lvl, elm_name, var_name, var_folder, dir_address):
    self.filename='%s %.0f percent %.2f s %.2f load'%(f_line.loc_name, f_loc, f_dur, load_lvl)
    elm = self.app.GetCalcRelevantObjects(elm_name)
    var = var_name
    res = self.app.GetFromStudyCase('*.ElmRes')
    for i_var in range(len(var)):
        self.comRes = self.app.GetFromStudyCase("ComRes")
        self.comRes.pResult = res
        self.comRes.iopt_exp = 6 # to export as csv
        self.comRes.f_name = dir_address+"\\"+"%s\\%s.csv"%(var_folder[i_var],self.filename) # File Name
        self.comRes.iopt_sep = 0 # to use not the system seperator
        self.comRes.col_Sep = r','
        self.comRes.dec_Sep = r'.'
        self.comRes.iopt_honly = 0 # to export data and not only the header
        self.comRes.iopt_csel = 1 # export only selected variables
        self.comRes.iopt_locn = 1
        self.comRes.ciopt_head = 0
        self.comRes.element = elm
        self.comRes.iopt_tsel = 1
        self.comRes.to = 30
        self.comRes.cfrom = 0
        variable = []
        for i_elm in range(len(elm)):
            variable.append(var[i_var])
        self.comRes.variable = variable
        self.comRes.Execute()

sim.getResult(
    f_loc_iter,
    f_line_iter,
    f_dur_iter,
    load_lvl_iter,
    elm_name = '*.ElmSym',
    var_name = [MONITORED_VARIABLES['*.ElmSym'][0]],
    var_folder = [VAR_FOLDER['*.ElmSym'][0]],
    dir_address = join(parentAddress,"csv"))

In [27]:
res.GetVariable(-1)

RuntimeError: method call failed

In [14]:
n_rows = res.GetNumberOfRows()
n_cols = res.GetNumberOfColumns()
res_var = [res.GetVariable(i) for i in range(-1, n_cols)]
res_var_desc = [res.GetDescription(i) for i in range(-1, n_cols)]
res_obj = [res.GetObject(i).loc_name for i in range(-1, n_cols)]
matrix_res = pd.DataFrame({'element': res_obj,
                           'variable': res_var,
                           'variable_description': res_var_desc
                           })
matrix_res
dict_res = {
    str(col + 1): [res.GetValue(row, col)[1] for row in range(n_rows)]
    for col in range(-1, n_cols)
}
pd.DataFrame(dict_res)
# res.GetVariable(4)
# for col in range(-1, n_cols):

# for i in range(n_rows):
#     print(res.GetValue(i, -1)[1])

KeyboardInterrupt: 

In [84]:
res.GetDescription(-1)

'Time'

In [20]:
import os
import itertools
import pandas as pd
from tqdm import tqdm
import time

# Initialize variables
dataset_subdirs = ['magnitude', 'angle', 'time', 'out_of_step']
dataset_dir = 'output/dataset'

# Create directories if they don't exist
for dir in dataset_subdirs:
    os.makedirs(f'{dataset_dir}/{dir}', exist_ok=True)

# Prepare an empty list to store iteration data
iteration_data = []

# Pre-compute total iterations
total_iterations = len(load_level) * len(f_line) * len(f_location) * len(f_duration)
num_digits = len(str(total_iterations))

# Start progress bar
with tqdm(total=total_iterations, desc="Simulating", unit="iter") as pbar:
    iteration_counter = 1
    for load_lvl_iter in load_level:
        # Perform load setup and OPF analysis
        sim.loadSetup(load_lvl_iter, load_default)
        try:
            sim.opfAnalysis('los')
        except AttributeError:
            sim.opfAnalysis('dev')
        
        # Perform short circuit setup and RMS simulation
        for f_line_iter, f_loc_iter, f_dur_iter in itertools.product(f_line, f_location, f_duration):
            # Start iteration timing
            iter_start_time = time.perf_counter()
            filename = f'scenario_{iteration_counter:0{num_digits}d}'

            # Jalakankan simulasi RMS
            sim.scSetup(f_loc_iter, f_line_iter, f_dur_iter)
            sim.rmsSimulation(MONITORED_VARIABLES, t_start=-100, t_step=10, t_stop=30)

            # Record the iteration time
            iter_end_time = time.perf_counter()
            iter_time = iter_end_time - iter_start_time

            # Save iteration data to the list
            iteration_data.append({
                "scenario": filename,
                "load_level": load_lvl_iter,
                "f_line": f_line_iter.loc_name,
                "f_location": f_loc_iter,
                "f_duration": f_dur_iter,
                "iter_time": iter_time
            })

            # Update tqdm progress bar
            pbar.set_postfix({
                "current_scenario": filename,
                "completed": f"{pbar.n+1}/{total_iterations}"
            })
            pbar.update(1)
            iteration_counter += 1
            

# After the loop finishes, create a DataFrame from the iteration data
df = pd.DataFrame(iteration_data)

# Optionally, save the DataFrame to a CSV file
df.to_csv('iteration_times.csv', index=False)

print("Simulation completed. Iteration times saved to 'iteration_times.csv'.")


Simulating:   0%|          | 0/8840 [00:00<?, ?iter/s]


AttributeError: 'PowerFactorySim' object has no attribute 'scSetup'

In [17]:
iter_time

0.0

In [4]:
for load_lvl_iter in load_level:
    filename_load = 'Kondisi %d persen beban'%(int(round(load_lvl_iter*100,0)))
    if exists(join(parentAddress,'csv',filename_load+'.xlsx')):
        print(filename_load+' exist')
    else:
        print(filename_load+' does not exist')
        sim.loadSetup(load_lvl_iter, load_ori)
        try:
            sim.opfAnalysis()
            sim.ldfAnalysis()
        except AttributeError:
            sim.ldfAnalysis()
        sim.saveLoadandGen(load_lvl_iter, join(parentAddress,'csv'))
    for f_line_iter, f_loc_iter, f_dur_iter in itertools.product(f_line, f_location, f_duration):
        time_start=perf_counter()
        k += 1
        filename = '%s %.0f percent %.2f s %.2f load'%(f_line_iter.loc_name, f_loc_iter, f_dur_iter, load_lvl_iter)
        if exists(join(parentAddress,'csv','rotor_angle',filename+'.csv')):
            print(filename+' exist')
        else:
            print(filename+' does not exist')
            
            sim.shortCircuitSetup(f_loc_iter, f_line_iter, f_dur_iter)
            sim.simulationRMS(MONITORED_VARIABLES, t_start=-100, t_step=10, t_stop=30)
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmSym',
                var_name = [MONITORED_VARIABLES['*.ElmSym'][0]],
                var_folder = [VAR_FOLDER['*.ElmSym'][0]],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmSym',
                var_name = [MONITORED_VARIABLES['*.ElmSym'][1]],
                var_folder = [VAR_FOLDER['*.ElmSym'][1]],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmRes',
                var_name = ['b:tnow'],
                var_folder = ['time'],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmTerm',
                var_name = [MONITORED_VARIABLES['*.ElmTerm'][0]],
                var_folder = [VAR_FOLDER['*.ElmTerm'][0]],
                dir_address = join(parentAddress,"csv"))

            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmTerm',
                var_name = [MONITORED_VARIABLES['*.ElmTerm'][1]],
                var_folder = [VAR_FOLDER['*.ElmTerm'][1]],
                dir_address = join(parentAddress,"csv"))
            
            print("%d / %d - %s %d percent %.2f s %.2f load"%(k,len(f_line)*len(f_location)*len(f_duration)*len(load_level),f_line_iter.loc_name,f_loc_iter,f_dur_iter,load_lvl_iter))
            time_end=perf_counter()
            minute,second=divmod((time_end-time_start),60)
            elapsed_hour, elapsed_sec = divmod((time_end-tstarttot), 3600)
            elapsed_min, elapsed_sec = divmod(elapsed_sec, 60)
            print("Progress %.2f persen - Waktu running per iterasi: %.2f detik - Waktu terlewati: %d jam %d menit %.2f detik"%(100*k/(len(f_line)*len(f_location)*len(f_duration)*len(load_level)),second,elapsed_hour,elapsed_min,elapsed_sec))
hour,second=divmod((time_end-tstarttot),3600)
minute,second=divmod(second,60)
print("Total waktu: %d jam %d menit %.2f detik"%(hour,minute,second))
sim.loadSetup(1, load_ori)

,name,parallel,V,pf,P,Q
0,G 01,1,1.015863,0.988703,602.284677,-91.308164
1,G 02,1,0.982000,0.966402,512.292188,136.254946
2,G 03,1,0.990396,0.998661,215.743747,11.177586
3,G 04,1,1.024547,0.999929,226.007788,2.685207
4,G 05,2,1.025169,0.990644,226.332616,31.179573
5,G 06,1,1.020712,0.999990,385.692937,-1.738357
6,G 07,1,1.022495,0.999879,168.582075,2.621380
7,G 08,1,1.012804,0.973286,205.317333,-48.433893
8,G 09,1,1.038461,0.934800,291.751331,-110.850112
9,G 10,1,1.007967,0.930295,221.584282,-87.370294
